# Temporal Understanding Through fMRI Analysis

Testing the hypothesis that temporal understanding requires physiological state mediation (A->B->C).

## Hypothesis:
World models cannot gain comprehensive temporal understanding solely from external observations.
Biometric data provides insight into human temporal understanding through physiological state transitions.

In [1]:
# Imports
%matplotlib inline
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import wandb
import logging
from pathlib import Path
from typing import Dict, Optional, Tuple

from learnedSpectrum.config import Config, DataConfig
from learnedSpectrum.models import VisionTransformerModel
from learnedSpectrum.train import train_loop, evaluate
from learnedSpectrum.visualization import TemporalUnderstandingVisualizer
from learnedSpectrum.physiological import PhysiologicalStateTracker
from learnedSpectrum.causal import CausalAnalysisModule
from learnedSpectrum.rl import TemporalStateEncoder
from learnedSpectrum.data import DatasetManager, create_dataloaders

In [2]:
# Configuration
config = Config()
data_config = DataConfig()

In [3]:
# Initialize wandb
wandb.init(
    project="temporal-understanding",
    config={
        "architecture": "vit-temporal",
        "dataset": "fmri-learning-stages",
        "epochs": config.NUM_EPOCHS,
        "batch_size": config.BATCH_SIZE,
        "learning_rate": config.LEARNING_RATE,
        "temporal_analysis": config.TEMPORAL_ANALYSIS,
        "causal_inference": config.CAUSAL_INFERENCE
    }
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: tawarner (tawarner-usc). Use `wandb login --relogin` to force relogin


In [4]:
# Device setup and model initialization
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [5]:
# Initialize models
model = VisionTransformerModel(config).to(device)
physiological_tracker = PhysiologicalStateTracker(config.EMBED_DIM).to(device)
causal_analyzer = CausalAnalysisModule(config.EMBED_DIM, config.TEMPORAL_DIM).to(device)

In [6]:
# Initialize optimizer with gradient clipping
optimizer = torch.optim.AdamW(model.parameters(), lr=config.LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=config.NUM_EPOCHS)
scaler = torch.amp.GradScaler('cuda') 

In [7]:
# Initialize visualization
vis_manager = TemporalUnderstandingVisualizer(config.VIS_DIR)

In [8]:
# Load and preprocess data
data_manager = DatasetManager(config, data_config)
train_ds, val_ds, test_ds = data_manager.prepare_datasets()
train_loader, val_loader, test_loader = create_dataloaders(train_ds, val_ds, test_ds, config)

Validating samples:   0%|          | 0/264 [00:00<?, ?file/s]

Analyzing timepoints:   0%|          | 0/184 [00:00<?, ?file/s]

Analyzing timepoints:   0%|          | 0/40 [00:00<?, ?file/s]

Analyzing timepoints:   0%|          | 0/40 [00:00<?, ?file/s]

In [9]:
# Training loop
model = train_loop(
    model=model,
    train_dl=train_loader,
    val_dl=val_loader, 
    optimizer=optimizer,
    config=config
)


Epoch 1/50


Training:   0%|                                                                                                                                    | 0/23 [00:00<?, ?it/s]C:\Users\twarn\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\_dynamo\eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
C:\Users\twarn\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Epoch 1/50
Train Loss: 2.4927
Val Loss: 1.7392
Val Accuracy: 0.2000
Val Balanced Accuracy: 0.2500

Prediction Distribution:
class_2: 0.975
class_3: 0.025

Label Distribution:
class_0: 0.350
class_1: 0.350
class_2: 0.200
class_3: 0.100
Epoch 1/50 - Train Loss: 2.4927, Val Loss: 1.7392, Val Acc: 0.2000

Epoch 2/50


Training:   0%|                                                                                                                                    | 0/23 [00:00<?, ?it/s]C:\Users\twarn\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\_dynamo\eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
C:\Users\twarn\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Epoch 2/50
Train Loss: 2.4413
Val Loss: 1.6383
Val Accuracy: 0.2000
Val Balanced Accuracy: 0.2500

Prediction Distribution:
class_2: 0.975
class_3: 0.025

Label Distribution:
class_0: 0.350
class_1: 0.350
class_2: 0.200
class_3: 0.100
Epoch 2/50 - Train Loss: 2.4413, Val Loss: 1.6383, Val Acc: 0.2000

Epoch 3/50


...

Epoch 25/50
Train Loss: 1.1861
Val Loss: 1.2133
Val Accuracy: 0.4000
Val Balanced Accuracy: 0.4777

Prediction Distribution:
class_1: 0.775
class_2: 0.025
class_3: 0.200

Label Distribution:
class_0: 0.350
class_1: 0.350
class_2: 0.200
class_3: 0.100
Epoch 25/50 - Train Loss: 1.1861, Val Loss: 1.2133, Val Acc: 0.4000

Epoch 26/50


Training:   0%|                                                                                     | 0/23 [00:00<?, ?it/s]C:\Users\twarn\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\_dynamo\eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
C:\Users\twarn\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Epoch 26/50
Train Loss: 1.2182
Val Loss: 1.2047
Val Accuracy: 0.2750
Val Balanced Accuracy: 0.2098

Prediction Distribution:
class_0: 0.475
class_1: 0.400
class_2: 0.125

Label Distribution:
class_0: 0.350
class_1: 0.350
class_2: 0.200
class_3: 0.100
Epoch 26/50 - Train Loss: 1.2182, Val Loss: 1.2047, Val Acc: 0.2750

Epoch 27/50


Training:   0%|                                                                                     | 0/23 [00:00<?, ?it/s]C:\Users\twarn\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\_dynamo\eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
C:\Users\twarn\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
# Evaluate and visualize results
test_loss, test_metrics = evaluate(
    model=model,
    test_dl=test_loader,
    config=config,
    physiological_tracker=physiological_tracker,
    causal_analyzer=causal_analyzer
)

In [ ]:
# Plot comprehensive results
vis_manager.plot_state_transitions(
    test_metrics['state_analysis'],
    save_name='final_state_transitions'
)

vis_manager.plot_temporal_understanding_analysis(
    test_metrics['temporal_results'],
    save_name='final_temporal_analysis'
)

In [ ]:
# Log final results to wandb
wandb.log({
    "test_accuracy": test_metrics['accuracy'],
    "test_loss": test_metrics['loss'],
    "test_temporal_consistency": test_metrics['temporal_consistency'],
    "causal_strength": test_metrics['causal_analysis']['mean_strength'],
    "ltc_stability": test_metrics['ltc_analysis']['stability_metric'],
    "temporal_understanding_score": test_metrics['temporal_understanding_score']
})

In [ ]:
# Save visualizations to wandb
wandb.log({
    "state_transitions": wandb.Image(str(Path(config.VIS_DIR) / 'final_state_transitions.png')),
    "temporal_analysis": wandb.Image(str(Path(config.VIS_DIR) / 'final_temporal_analysis.png'))
})